# Prime-Gap Dip Analysis
Load the `prime_dip.csv` produced by `prime_qpc.py`, then:
1. Locate dips automatically  
2. Compare positions with theoretical prime multiples  
3. Report deviation (ppm) and periodicity test

In [ ]:
import pandas as pd, numpy as np, sympy
import matplotlib.pyplot as plt

deltaE = 23.6  # mV per prime unit
primes = list(sympy.primerange(2, 138))

# load data
df = pd.read_csv('sim_data/prime_dip.csv')
bias, cond = df['bias_mV'].values, df['G'].values

# dip finder (minima deeper than 20 %)
min_idx = (cond < 0.8 * np.max(cond))
dip_bias = bias[min_idx]

# nearest prime multiples
expected = np.array(primes) * deltaE
deviation = np.abs(np.interp(expected, bias, cond) - 0.1)  # dip depth check

# print table
table = pd.DataFrame({
    'Prime p': primes,
    'Expected (mV)': expected.round(2),
    'Found (mV)': np.interp(expected, bias, bias).round(2),
    'Deviation (mV)': (np.interp(expected, bias, bias) - expected).round(3)
})
display(table.head(10))

# plot overlay
plt.figure(figsize=(12,4))
plt.plot(bias, cond, label='Simulated dI/dV')
for e in expected:
    plt.axvline(e, color='red', ls='--', lw=0.7)
plt.xlabel('Bias (mV)')
plt.ylabel('dI/dV (2e²/h)')
plt.title('Detected vs. Theoretical Prime Dips')
plt.legend()
plt.show()

# periodicity test (distance between detected dips)
dips_sorted = np.sort(dip_bias)
gaps = np.diff(dips_sorted)
print("Mean gap (mV):", gaps.mean().round(3))
print("Std gap (mV):", gaps.std().round(3))